In [27]:
import numpy as np
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import LineString
import sys

sys.path.append('../')
import road
from overpass import fetch_overpass, get_overpass_query
from elevation import get_elevation_from_srtm, calc_incline
from typing import *

In [28]:

columns = ['highway', 'maxspeed', 'lanes', 'name', 'oneway', 'surface']
highway_list = ["motorway", "motorway_link", "trunk", "trunk_link", "primary", "primary_link", 
                  "secondary", "secondary_link", "tertiary", "tertiary_link", "residential","cycleway"]
cycleway_list = None
cycleway_columns = ['cycleway:both', 'cycleway:left','cycleway:right']
if "cycleway" in highway_list:
    cycleway_list = ["lane", "opposite", "opposite_lane", "track", "opposite_track", 
                     "share_busway", "opposite_share_busway", "shared_lane",]
    columns += cycleway_columns
    columns += ['cycleway']
    
    #'oneway:bicycle','cycleway:left:oneway','cycleway:right:oneway',

In [29]:
columns

['highway',
 'maxspeed',
 'lanes',
 'name',
 'oneway',
 'surface',
 'cycleway:both',
 'cycleway:left',
 'cycleway:right',
 'cycleway']

In [30]:
bbox = [45.516012863655845,-73.61165474010419,45.54058887207495,-73.56153948806578]
bbox = [45.45, -73.6492855702676, 45.644709880535515, -73.44752816469271]
bbox = (*bbox,)

In [31]:
overpass_query = get_overpass_query(bbox,highway_list,cycleway_list)

In [32]:
fetch_overpass(overpass_query,columns,'')

OVERPASS Request ...
Convert to GeoPandas ...


/home/boivin/.pyenv/versions/quetzal_env/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


Write (way.geojson) ...


In [33]:
links, nodes = road.get_links_and_nodes('way.geojson', split_direction=True)
nodes = nodes.set_crs(links.crs)

In [34]:
links = rename_bicycle_tags(links,'cycleway')
links = rename_bicycle_tags(links,'cycleway:both')
links = rename_bicycle_tags(links,'cycleway:left')
links = rename_bicycle_tags(links,'cycleway:right')


links['combine_cycle_tag'] = links['cycleway'] +' '+ \
                                links['cycleway:both'] +' '+ \
                                links['cycleway:left'] +' '+ \
                                links['cycleway:right'] 


# simple method. everything with a tag highway is an highway both side. using the road oneway.
bike_dict={}
for string in links['combine_cycle_tag'].unique():
    val = string.split(' ')
    if val[0]=='yes':
        bike_dict[string]='yes'
    elif val[0] == 'shared':
        bike_dict[string]='shared'
    elif 'yes' in val[1:]:
        bike_dict[string]='yes'
    elif  'share' in val[1:]:
        bike_dict[string]='share'
    else :
        bike_dict[string]='no'
links['cycleway'] = links['combine_cycle_tag'].apply(lambda x: bike_dict.get(x))
links.loc[links['highway']=='cycleway','cycleway'] = 'yes'

['no', 'yes', 'shared']
['no', 'yes', 'shared']
['no', 'yes', 'shared']
['no', 'yes', 'shared']


In [35]:
#links.to_file('b_links.geojson',drivers='GeoJSON')
#nodes.to_file('b_nodes.geojson',drivers='GeoJSON')

In [36]:
links = links.drop(columns = cycleway_columns)
#remove highway not asked for. (because of cycleway)
links = links[links['highway'].isin(highway_list)]

In [37]:
links

,id,type,highway,maxspeed,lanes,name,oneway,surface,cycleway,a,b,geometry,combine_cycle_tag
road_link_0,4320462,way,trunk,70,4,Autoroute Ville-Marie,yes,asphalt,no,road_node_12069,road_node_15880,"LINESTRING (-73.55832 45.50704, -73.55806 45.5...",no no no no
road_link_1,4320471,way,residential,30,1,Rue Mackay,yes,asphalt,no,road_node_8703,road_node_8773,"LINESTRING (-73.57878 45.49667, -73.57870 45.4...",no no no no
road_link_2,4320471,way,residential,30,1,Rue Mackay,yes,asphalt,no,road_node_8773,road_node_11517,"LINESTRING (-73.57870 45.49663, -73.57867 45.4...",no no no no
road_link_3,4320471,way,residential,30,1,Rue Mackay,yes,asphalt,no,road_node_11517,road_node_10816,"LINESTRING (-73.57705 45.49584, -73.57696 45.4...",no no no no
road_link_4,4320473,way,tertiary,40,1,Rue du Fort,yes,asphalt,no,road_node_15803,road_node_7237,"LINESTRING (-73.58373 45.49316, -73.58366 45.4...",no no no no
...,...,...,...,...,...,...,...,...,...,...,...,...,...
road_link_60383,1190379610,way,cycleway,NaN,NaN,longueuil ouest-est,False,asphalt,yes,road_node_25064,road_node_1453,"LINESTRING (-73.49686 45.54346, -73.49682 45.5...",yes no no no
road_link_60384,1190379611,way,cycleway,NaN,NaN,longueuil ouest-est,no,asphalt,yes,road_node_11932,road_node_2695,"LINESTRING (-73.49482 45.54554, -73.49421 45.5...",no no no no
road_link_60385,1190379612,way,cycleway,NaN,NaN,longueuil ouest-est,False,asphalt,yes,road_node_12367,road_node_9599,"LINESTRING (-73.49495 45.54544, -73.49489 45.5...",yes no no no
road_link_60386,1190424273,way,motorway,50,3,Autoroute Bonaventure,yes,asphalt,no,road_node_26760,road_node_10259,"LINESTRING (-73.55579 45.49568, -73.55565 45.4...",no no no no


In [38]:
bike_links = links[links['cycleway']!='no'].copy()

bike_links['abset'] = [frozenset(el) for el in zip(bike_links['a'], bike_links['b'])]
abset = bike_links.groupby('abset')[['a']].agg(len)
oneway_abset = abset[abset['a']>1].index.values

bike_links['oneway'] = False
bike_links.loc[bike_links['abset'].isin(oneway_abset),'oneway'] = True

oneway_dict = bike_links['oneway'].to_dict()
links.loc[links['cycleway']!='no','oneway'] = bike_links['oneway']

In [1]:
test=[1,2,3]

In [44]:
'''
links = rename_bicycle_tags(links,'oneway:bicycle')
    links = rename_bicycle_tags(links,'cycleway:left:oneway')
    links = rename_bicycle_tags(links,'cycleway:right:oneway')

    links['combine_oneway_tag'] = links['oneway:bicycle'] +'-'+ \
                                    links['cycleway:left:oneway']+'-'+ \
                                    links['cycleway:right:oneway'] 
'''

"\nlinks = rename_bicycle_tags(links,'oneway:bicycle')\n    links = rename_bicycle_tags(links,'cycleway:left:oneway')\n    links = rename_bicycle_tags(links,'cycleway:right:oneway')\n\n    links['combine_oneway_tag'] = links['oneway:bicycle'] +'-'+                                     links['cycleway:left:oneway']+'-'+                                     links['cycleway:right:oneway'] \n"

In [45]:
links = road.clean_oneway(links)

In [46]:
links = road.clean_maxspeed(links)

In [47]:
links = road.drop_duplicated_links(links)

764 links dropped


In [48]:
links = road.simplify(links,cutoff=10)
len(links)

20985 deg 2 nodes
find path with large cutoff for 1  origins
find path with large cutoff for 1  origins
find path with large cutoff for 1  origins
find path with large cutoff for 2  origins
0 links were not merge because the oneway field is not the same
0 links were not merge because the highway field is not the same
0 merged_links unmerged because the geometry became a multilinestring


55066

In [87]:
df=links.copy()

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 16 µs


In [113]:
%%time
get_columns_with_list(df)

CPU times: user 88.9 ms, sys: 3.94 ms, total: 92.9 ms
Wall time: 91.2 ms


['id', 'maxspeed', 'lanes', 'name', 'surface', 'cycleway', 'combine_cycle_tag']

In [545]:
links = road.split_oneway(links)
len(links)

98782

In [546]:
links = road.main_strongly_connected_component(links,None,False)
len(links)

97825

In [547]:
links['maxspeed'] = links['maxspeed'].apply(lambda x: process_list_in_col(x,float,np.nanmean))
links['lanes'] = links['lanes'].apply(lambda x: process_list_in_col(x,float,lambda x: np.floor(np.nanmean(x))))

../road.py:24: RuntimeWarning: Mean of empty slice
  return  function([new_type(val) for val in col_values])


In [548]:
links['cycleway'] = links['cycleway'].apply(lambda x: process_list_in_col(x,str,lambda x: np.sort(x)[-1]))

In [549]:
for col in ['highway','name','surface']:
    links[col] = links[col].apply(lambda x: remove_list_in_col(x,'first'))

In [550]:
links = fill_na_col(links,'highway','maxspeed',np.mean)
links = fill_na_col(links,'highway','lanes',lambda x: np.floor(np.mean(x)))

In [551]:

# Add length
print("Write Links and Nodes ...")
epsg = get_epsg(nodes.iloc[0]['geometry'].y, nodes.iloc[0]['geometry'].x)
links['length'] = links.to_crs(epsg).length

# Add Time
links['time'] = links['length']/(links['maxspeed']*1000/3600)
links = links.rename(columns = {'maxspeed' : 'speed'})

# reindex and remove ununsed nodes
links = links.reset_index(drop=True)
links.index = 'road_link_'+links.index.astype(str)
nodes_set = set(links['a']).union(set(links['b']))
nodes = nodes.loc[list(nodes_set)].sort_index()

Write Links and Nodes ...


In [552]:
print('Adding elevation')
el_dict = get_elevation_from_srtm(nodes)
nodes['elevation'] = nodes.index.map(el_dict.get)
# incline from node a to b in deg. neg if going down (if b is lower dans a)
links['incline'] = calc_incline(links['a'].apply(lambda x: el_dict.get(x)).values,
                            links['b'].apply(lambda x: el_dict.get(x)).values,
                            links['length'].values)


Adding elevation
file save to /tmp


In [4]:
links.to_file('road_links.geojson')
nodes.to_file('road_nodes.geojson')

NameError: name 'links' is not defined

In [16]:
from road import *
from bike import *
from elevation import *

In [19]:
links, nodes = get_links_and_nodes('way.geojson', split_direction=False)
nodes = nodes.set_crs(links.crs)
links = test_bicycle_process(links,cycleway_columns,highway_list)
links = clean_oneway(links)
links = clean_maxspeed(links)
links = drop_duplicated_links(links, sort_column='maxspeed')
links = simplify(links)
links = split_oneway(links)
links = main_strongly_connected_component(links,None,False)

print('removing list in columns ...')
links['maxspeed'] = links['maxspeed'].apply(lambda x: process_list_in_col(x,float,np.nanmean))
links['lanes'] = links['lanes'].apply(lambda x: process_list_in_col(x,float,lambda x: np.floor(np.nanmean(x))))
if 'cycleway' in links.columns:
    links['cycleway'] = links['cycleway'].apply(lambda x: process_list_in_col(x,str,lambda x: np.sort(x)[-1]))
for col in ['id', 'type', 'highway','name','surface']:
    links[col] = links[col].apply(lambda x: remove_list_in_col(x,'first'))


# Fill NaN with mean values by highway
links = fill_na_col(links, 'highway', 'maxspeed', np.mean)
links = fill_na_col(links, 'highway', 'lanes', lambda x: np.floor(np.mean(x)))

# Add length
print("Write Links and Nodes ...")
epsg = get_epsg(nodes.iloc[0]['geometry'].y, nodes.iloc[0]['geometry'].x)
links['length'] = links.to_crs(epsg).length

# Add Time
links['time'] = links['length']/(links['maxspeed']*1000/3600)
links = links.rename(columns = {'maxspeed' : 'speed'})

# reindex and remove ununsed nodes
links = links.reset_index(drop=True)
links.index = 'road_link_'+links.index.astype(str)
nodes_set = set(links['a']).union(set(links['b']))
nodes = nodes.loc[list(nodes_set)].sort_index()


['no', 'yes', 'shared']
['no', 'yes', 'shared']
['no', 'yes', 'shared']
['no', 'yes', 'shared']
182 links dropped
18297 deg 2 nodes
find path with large cutoff for 2  origins
find path with large cutoff for 1  origins
find path with large cutoff for 1  origins
find path with large cutoff for 2  origins
find path with large cutoff for 6  origins
0 links were not merge because the oneway field is not the same
0 links were not merge because the highway field is not the same
0 merged_links unmerged because the geometry became a multilinestring
removing list in columns ...


../road.py:24: RuntimeWarning: Mean of empty slice
  return  function([new_type(val) for val in col_values])


Write Links and Nodes ...


In [20]:
print('Adding elevation')
el_dict = get_elevation_from_srtm(nodes)
nodes['elevation'] = nodes.index.map(el_dict.get)
# incline from node a to b in deg. neg if going down (if b is lower dans a)
links['incline'] = calc_incline(links['a'].apply(lambda x: el_dict.get(x)).values,
                            links['b'].apply(lambda x: el_dict.get(x)).values,
                            links['length'].values)

Adding elevation
file save to /tmp


In [26]:
links.to_file('road_links.geojson')
nodes.to_file('road_nodes.geojson')

In [25]:
links = links.drop(columns='combine_cycle_tag')

In [3]:
links

NameError: name 'links' is not defined